In [1]:
import os
from audiofile.core.info import channels 
import opensmile
import audiofile
import numpy as np
import pandas as pd

SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


In [2]:
# Function to extract audio features from segments with assistance  and adaptation from George's DyMand code

# Get start and end times for speaker annotations
def get_starts_ends(file,gender,data_input=None):
    data = data_input
    if data_input is None:
        data = [i.strip('\n').split('\t') for i in open(file)]
    times = []
    for i in data:
        if i[-1]==gender:
            start = i[0]
            end = i[1]
            times.append((float(start),float(end)))
    return (times)

def extract_audio(num_of_channels, audio_file, start, duration):
    # Check if audio has 2 channels
    if num_of_channels == 2:
        signal, sampling_rate = audiofile.read(audio_file, duration=duration,  offset=start)
        signal_1 = signal[0,:]
        signal_2 = signal[1,:]
        return signal_1, signal_2, sampling_rate
    else:
        signal, sampling_rate = audiofile.read(audio_file, duration=duration,  offset=start)
        return signal, sampling_rate

def combine_audio_segments(start_end_times,audio_file):
    if (len(start_end_times) == 0):
        print("There are no segments to combine")
        return [],-1

    num_of_channels = audiofile.channels(audio_file)
    audio_signal = []
    audio_signal_1 = []
    audio_signal_2 = []
    final_audio_signal = []
    sampling_rate = -1
    total_durations = []
    for i,(start,end) in enumerate(start_end_times):
        duration = end - start
        total_durations.append(duration)

        # Check if audio has 2 channelss
        if num_of_channels == 2:
            signal_1, signal_2, sampling_rate =  extract_audio(num_of_channels, audio_file, start, duration)
            audio_signal_1.extend(signal_1)
            audio_signal_2.extend(signal_2) 

        else:
            signal,sampling_rate = extract_audio(num_of_channels, audio_file, start, duration)
            audio_signal.extend(signal)

    return final_audio_signal, sampling_rate

def extract_audio_features(audio_signal, sampling_rate):
    # Check if list is empty
    if (len(audio_signal) == 0):
        print("There are no audio data")
        return None

    # Check number of channels
    num_of_channels = -1
    shape_len = len(audio_signal.shape)

    if shape_len == 1:
        num_of_channels = 1
    else:
        num_of_channels = audio_signal.shape[0]

    smile = opensmile.Smile(
        feature_set=opensmile.FeatureSet.eGeMAPSv02, 
        feature_level=opensmile.FeatureLevel.Functionals,
        num_channels=num_of_channels) #latest version advised, Functionals level give 88 features we want
    features = smile.process_signal(audio_signal,sampling_rate)
    return(features)    #returns a data frame



## Load paths


In [3]:
meta = pd.read_csv("align_overview_sum - align_overview_sum.csv")
# Filter data 

# Conditions: 

# contains speech = yes
# female speak or male speek != no, no
# annotate = 1
# Arousal_binary != -1
# Valence_binary != -1
# check date != 0 (out-of-hours recording)

data = meta.loc[(meta['Arousal_binary'] != -1) & (meta['Valence_binary'] != -1) & (meta['check date'] != 0)].reset_index(drop=True)
data['ID'] = data['audio'].str[:5]
fspeak = ["P_001","P_002","P_003","P_004","P_005","P_006","P_007","P_008","P_009","P_010","P_011","Z_012","P_013"]
mspeak = ["Z_001","Z_002","Z_003","Z_004","Z_005","Z_006","Z_007","Z_008","Z_009","Z_010","Z_011","P_012","Z_013"]
data = data.loc[(data["female speak"] == "yes") & (data["male speak"] == "yes")].reset_index(drop=True)

# Create complete data paths
p_path = "Z:\DyMand Data\Field data\Smartwatch\Partner_in\\"
z_path = "Z:\DyMand Data\Field data\Smartwatch\Zielperson\\"

data['root_dir'] = np.where(data['audio'].str.startswith("P"), p_path, z_path)
data['abs_path'] = data['root_dir'] + data['audio']

# subdir where data exists
data_paths_audio = data['abs_path'].tolist()


(380, 25)


In [5]:
# Annotations path

data['ID'] = data['audio'].str[:5].str.replace("_","") + "\\"
data['Hours'] = data['audio'].str.split("\\").str[1:3].str.join("-")+"\Annotations"
data['abs_path_annotations'] = "Z:\Annotations\Annotated\\" + data['ID'] + data['Hours'] + "\\Label Track.txt"

# subdir where data exists in list format
data_paths_annotations = data['abs_path_annotations'].tolist()


In [6]:
def get_all_audio_features(data_paths_audio, data_paths_annotations, data_overview_file):
    genders = pd.read_csv(data_overview_file)
    feat = []
    feature_names = []
    skips_label = []
    # Go through all the labels
    
    for audio_path, annot_path in zip(data_paths_audio, data_paths_annotations):
        for file in os.listdir(audio_path):
            if file == "watchRecordAudio.wav":
                filepath = audio_path + os.sep + file # get file
                
                split_list = filepath.split("\\")
                split_time = split_list[-3].split("-")
                couple_id = split_list[5][-2:]
                name = "_".join(split_list[5:8])
                name_transcript = "_".join(split_list[5:9])
                
                # Get annotation info
                annotations = annot_path
                audio_file = filepath
                
                print()
                print("Audio path: {}".format(audio_file))

                # Get start and end times
                try: 
                    times_f = get_starts_ends(annotations,'f')
                    times_m = get_starts_ends(annotations,'m')
                    
                except FileNotFoundError:
                    print("Get starts function could not find annotations")
                    skips_label.append(audio_file)

                # Combine audio segments

                audio_signal_f, sampling_rate_f = combine_audio_segments(times_f,audio_file)
                audio_signal_m, sampling_rate_m = combine_audio_segments(times_m,audio_file)


                # Extract features
                feature_vector_f = extract_audio_features(audio_signal_f,sampling_rate_f)
                feature_vector_m = extract_audio_features(audio_signal_m,sampling_rate_m)

                # Check if features were extracted
                if feature_vector_f is not None:
                    features_f = [couple_id,name,'f',name_transcript]
                    features_f.extend(feature_vector_f.to_numpy()[0,:])
                    feat.append(features_f)
                    feature_names = feature_vector_f.columns

                if feature_vector_m is not None:
                    features_m = [couple_id,name,'m',name_transcript]
                    features_m.extend(feature_vector_m.to_numpy()[0,:])
                    feat.append(features_m)
   
    columns = ['ID', 'Name', 'Gender','Name Transcript']+list(feature_names)
    return pd.DataFrame(feat, columns = [columns]), skips_label

In [7]:
df,skips_label = get_all_audio_features(data_paths_audio, data_paths_annotations, "Couples Data Overview.csv")


Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_001\Day_2\Hour_16\07-16-2019 16_44_00\watchRecordAudio.wav


C:\Users\madha\anaconda3\lib\site-packages\audobject\core\decorator.py:113: UserWarning: 'num_channels' argument is deprecated and will be removed with version 0.13.0. Use 'channels' instead.
  func(self, *args, **kwargs)



Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_001\Day_2\Hour_18\07-16-2019 18_24_27\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_001\Day_2\Hour_9\07-16-2019 09_00_26\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_001\Day_3\Hour_16\07-17-2019 16_06_50\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_001\Day_3\Hour_17\07-17-2019 17_02_08\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_001\Day_3\Hour_19\07-17-2019 19_09_54\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_001\Day_3\Hour_9\07-17-2019 09_01_14\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_003\Day_2\Hour_19\11-19-2019 19_01_25\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_003\Day_2\Hour_20\11-19-2019 20_02_31\watchRecordAudio.wav

Audio path: Z:\DyMand Dat


Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_007\Day_0\Hour_16\02-16-2020 16_02_42\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_007\Day_0\Hour_17\02-16-2020 17_35_39\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_007\Day_0\Hour_18\02-16-2020 18_11_33\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_007\Day_0\Hour_22\02-16-2020 22_15_40\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_007\Day_0\Hour_9\02-16-2020 09_44_00\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_007\Day_1\Hour_22\02-10-2020 22_21_15\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_007\Day_1\Hour_9\02-10-2020 09_07_52\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_007\Day_2\Hour_19\02-11-2020 19_08_01\watchRecordAudio.wav

Audio path: Z:\DyMand Dat


Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_010\Day_0\Hour_11\02-16-2020 11_03_08\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_010\Day_0\Hour_12\02-16-2020 12_32_00\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_010\Day_0\Hour_13\02-16-2020 13_01_53\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_010\Day_0\Hour_16\02-16-2020 16_17_26\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_010\Day_0\Hour_17\02-16-2020 17_04_01\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_010\Day_0\Hour_21\02-16-2020 21_12_50\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_010\Day_0\Hour_9\02-16-2020 09_44_00\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_010\Day_1\Hour_18\02-10-2020 18_00_45\watchRecordAudio.wav

Audio path: Z:\DyMand Da

There are no audio data

Audio path: Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_003\Day_6\Hour_12\11-23-2019 12_00_44\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_003\Day_6\Hour_13\11-23-2019 13_15_40\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_003\Day_6\Hour_14\11-23-2019 14_44_15\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_003\Day_6\Hour_9\11-23-2019 09_04_16\watchRecordAudio.wav
There are no segments to combine
There are no audio data

Audio path: Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_004\Day_0\Hour_10\12-01-2019 10_02_13\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_004\Day_0\Hour_12\12-01-2019 12_22_21\watchRecordAudio.wav
There are no segments to combine
There are no audio data

Audio path: Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_004\Day_0\Hour_15\12-01-2019 15_30_59\watchRecordAudio.wav

Audio pa


Audio path: Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_007\Day_2\Hour_21\02-11-2020 21_35_16\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_007\Day_2\Hour_22\02-11-2020 22_44_20\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_007\Day_2\Hour_8\02-11-2020 08_44_18\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_007\Day_2\Hour_9\02-11-2020 09_10_03\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_007\Day_3\Hour_19\02-12-2020 19_06_29\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_007\Day_3\Hour_20\02-12-2020 20_00_32\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_007\Day_3\Hour_21\02-12-2020 21_06_35\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_007\Day_3\Hour_22\02-12-2020 22_01_27\watchRecordAudio.wav

Audio path: Z:\DyMand Dat


Audio path: Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_010\Day_0\Hour_17\02-16-2020 17_04_01\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_010\Day_0\Hour_18\02-16-2020 18_23_54\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_010\Day_0\Hour_19\02-16-2020 19_44_00\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_010\Day_0\Hour_20\02-16-2020 20_44_00\watchRecordAudio.wav
There are no segments to combine
There are no audio data

Audio path: Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_010\Day_0\Hour_21\02-16-2020 21_12_49\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_010\Day_1\Hour_18\02-10-2020 18_00_44\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_010\Day_1\Hour_20\02-10-2020 20_13_28\watchRecordAudio.wav

Audio path: Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_010\Day_1\Hour_21\02-10-20

In [8]:
df.to_csv("audio_features/AudioFeatures.csv", index=False)

## Code below for error checking - no need to run

In [14]:
# Sometimes data is recorded for the other person since they are in close proximity. We want to remove these
df['ID'] = df['Name'].str.split("_").str[:2].str.join("_")
audio_original_filter = df.drop(df[(df['Gender'] == "f") & (df['ID'].str.startswith("P")) & (df['ID'].str.startswith("P_012") != True)].index)
audio_original_filter_final = audio_original_filter.drop(audio_original_filter[(audio_original_filter['Gender'] == "m") & (audio_original_filter['ID'].str.startswith("Z")) & (audio_original_filter['ID'].str.startswith("Z_012") != True)].index)
audio_original_filter_final = audio_original_filter_final.reset_index(drop=True)

AttributeError: 'DataFrame' object has no attribute 'str'

In [10]:
fspeak = ["P_001","P_002","P_003","P_004","P_005","P_006","P_007","P_008","P_009","P_010","P_011","Z_012","P_013"]
mspeak = ["Z_001","Z_002","Z_003","Z_004","Z_005","Z_006","Z_007","Z_008","Z_009","Z_010","Z_011","P_012","Z_013"]
df_filtered = df.loc[(df['ID'].isin(fspeak)) & (df["Gender"] == "f") | (df['ID'].isin(mspeak)) & (df["Gender"] == "m")].reset_index(drop=True)

ValueError: Cannot index with multidimensional key

In [11]:
skips_label # These files do not have annotations

['Z:\\DyMand Data\\Field data\\Smartwatch\\Zielperson\\Z_010\\Day_6\\Hour_17\\02-15-2020 17_34_22\\watchRecordAudio.wav']

In [19]:
skipped_annotations = pd.DataFrame(skips_label)
skipped_annotations.to_csv("Labels without annotation files.csv",index=False)